In [16]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

In [1]:
## Arxiv 
## Tool creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [5]:
tools = [wiki,arxiv]

In [6]:
## Create the custom tools[RAG TOOL]

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())  
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1092912d0>, search_kwargs={})

In [18]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search", "Search any Information about Langsmith")
retriever_tool.name

'langsmith-search'

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/parthabhang/Desktop/GenAi Project/Search Engine Project/seenv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300))]

In [20]:
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [21]:
from langchain_groq import ChatGroq

In [22]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [24]:
## Prompt Template

from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
## Agent

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [26]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was found
Invoking: `arxiv` with `{'query': 'Langsmith'}`


Published: 2020-10-09
Title: Langsmith: An Interactive Academic Text Revision System
Authors: Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, Kentaro Inui
Summary: Despite the current diversity and inclusion initiatives in the academic
community, researchers with a non-native command Based on the result yielded by the tool call, I can provide a concise summary about Langsmith:

Langsmith is an interactive academic text revision system that aims to assist researchers with non-native language command to produce high-quality academic texts.

> Finished chain.


{'input': 'Tell me about langsmith',
 'output': 'Based on the result yielded by the tool call, I can provide a concise summary about Langsmith:\n\nLangsmith is an interactive academic text revision system that aims to assist researchers with non-native language command to produce high-quality academic texts.'}

In [29]:
agent_executor.invoke({"input":"Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as well as
Simons Foundation, Berkeley.Machine learning is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. The training data is used to adjust the algorithm's parameters to improve its performance on a specific task. Machine lea

{'input': 'Tell me about machine learning',
 'output': "Machine learning is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. The training data is used to adjust the algorithm's parameters to improve its performance on a specific task. Machine learning has many applications, including image and speech recognition, natural language processing, and personalized product recommendations.\n\nThe development of machine learning algorithms typically involves several steps, including:\n\n1. Data preprocessing: Cleaning and preparing the data for use in the algorithm.\n2. Model selection: Choosing the type of algorithm to use and the parameters to tune.\n3. Training: Using the data to adjust the algorithm's parameters to improve its performance.\n4. Evaluation: Testing the algorithm's performance on a separate dataset to ensure it generalizes well.\n5. Deployment: Integrating the trai

In [30]:
agent_executor.invoke({"input":"What is the paper 1706.03762 all about"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks 
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the transformer architecture, which has since become widely used in many NLP applications.

> Finished chain.


{'input': 'What is the paper 1706.03762 all about',
 'output': 'The paper "Attention Is All You Need" introduced the transformer architecture, which has since become widely used in many NLP applications.'}